<a href="https://colab.research.google.com/github/rkrissada/100DayOfMLCode/blob/master/day_056_customer_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("data.csv")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom



It took a few monutes to load the data, so I keep a copy as a backup.

In [0]:
df1 = df


We need to do santity check about the data

In [3]:
df1.Country.nunique()

19

In [4]:
df1.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'United '], dtype=object)

In [5]:
customer_country=df1[['Country','CustomerID']].drop_duplicates()

customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False)

,Country,CustomerID
18,United Kingdom,604
6,Germany,16
5,France,10
14,Portugal,5
15,Spain,2
8,Italy,2
0,Australia,2
4,EIRE,2
9,Japan,2
7,Iceland,1


More than 90% of the customers in the data are from United Kingdom, There’s some research indicating that customer clusters vary by geography, so here I’ll restrict the data to United Kingdom only.

In [0]:
df1 = df1.loc[df1['Country'] == 'United Kingdom']

Check whether there are missing values in each column.

There are 8947 missing values in CustomerID column, since our analysis is based on customers, we will remove these missing values.

In [7]:
df1.isnull().sum(axis=0)

InvoiceNo         0
StockCode         0
Description     111
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID     8947
Country           0
dtype: int64

In [8]:
df1 = df1[pd.notnull(df1['CustomerID'])]
df1.isnull().sum(axis=0)

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

Check the min and max values in Unit price column

In [9]:
df1.UnitPrice.min()

0.03

In [10]:
df1.Quantity.min()

-9360


Remove the negative values in Quantity column

In [11]:
df1 = df1[(df1['Quantity']>0)]
df1.Quantity.min()

1

After cleaning up, we now dealing with 14425 rows and 8 columns

In [12]:
df1.shape

(14425, 8)

In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14425 entries, 0 to 24289
Data columns (total 8 columns):
InvoiceNo      14425 non-null object
StockCode      14425 non-null object
Description    14425 non-null object
Quantity       14425 non-null int64
InvoiceDate    14425 non-null object
UnitPrice      14425 non-null float64
CustomerID     14425 non-null float64
Country        14425 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 1014.3+ KB


Check unique value for each column

In [14]:
def unique_counts(df1):
   for i in df1.columns:
       count = df1[i].nunique()
       print(i, ": ", count)
unique_counts(df1)

InvoiceNo :  776
StockCode :  2006
Description :  2011
Quantity :  91
InvoiceDate :  711
UnitPrice :  109
CustomerID :  563
Country :  1


In [15]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom



Add a column for total price

In [16]:
df1['TotalPrice'] = df1['Quantity'] * df1['UnitPrice']
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


Find out first and last order date in the data

In [17]:
df1['InvoiceDate'].min()

'12/1/2010 10:19'

In [18]:
df1['InvoiceDate'].max()

'12/9/2010 9:49'

Since recency is calculated for a point in time. The last invoice date is 9/12/2010, this is the date we will use to calculate recency.

In [0]:
import datetime as dt
NOW = dt.datetime(2010,12,9)

In [0]:
df1['InvoiceDate'] = pd.to_datetime(df1['InvoiceDate'])

Create a RFM table

In [0]:
rfmTable = df1.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, # Recency
                                        'InvoiceNo': lambda x: len(x),      # Frequency
                                        'TotalPrice': lambda x: x.sum()}) # Monetary Value

rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'TotalPrice': 'monetary_value'}, inplace=True)


Calculate RFM metrics for each customer

In [27]:
rfmTable.head()

,recency,frequency,monetary_value
CustomerID,,,
12747.0,3,7,358.56
12748.0,-2,544,3418.33
12826.0,-1,9,155.00
12838.0,7,59,390.79
12839.0,1,22,402.61


In [29]:
first_customer = df1[df1['CustomerID']== 12747.0]
first_customer

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
9754,537215,85124C,GREEN JUICY FRUIT PHOTO FRAME,12,2010-12-05 15:38:00,2.55,12747.0,United Kingdom,30.60
9755,537215,85124B,BLUE JUICY FRUIT PHOTO FRAME,6,2010-12-05 15:38:00,2.55,12747.0,United Kingdom,15.30
9756,537215,84879,ASSORTED COLOUR BIRD ORNAMENT,16,2010-12-05 15:38:00,1.69,12747.0,United Kingdom,27.04
9757,537215,85062,PEARL CRYSTAL PUMPKIN T-LIGHT HLDR,24,2010-12-05 15:38:00,1.65,12747.0,United Kingdom,39.60
9758,537215,85064,CREAM SWEETHEART LETTER RACK,6,2010-12-05 15:38:00,5.45,12747.0,United Kingdom,32.70
9759,537215,82484,WOOD BLACK BOARD ANT WHITE FINISH,36,2010-12-05 15:38:00,5.55,12747.0,United Kingdom,199.80
9760,537215,21136,PAINTED METAL PEARS ASSORTED,8,2010-12-05 15:38:00,1.69,12747.0,United Kingdom,13.52


In [34]:
(NOW - dt.datetime(2010,12,5)).days==4

True

The easies way to split metrics into segments is by using quartile.

1.  This gives us a starting point for detailed analysis

2. 4 segments are easy to understand and explain

In [35]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles

,recency,frequency,monetary_value
0.25,0.0,9.0,181.57
0.50,2.0,16.0,305.45
0.75,5.0,31.5,447.95


In [36]:
quantiles = quantiles.to_dict()
quantiles

{'frequency': {0.25: 9.0, 0.5: 16.0, 0.75: 31.5},
 'monetary_value': {0.25: 181.56999999999996,
  0.5: 305.45000000000005,
  0.75: 447.95},
 'recency': {0.25: 0.0, 0.5: 2.0, 0.75: 5.0}}

Create a segmented RFM table

In [0]:
segmented_rfm = rfmTable

Lowest recency, highest frequency and monetary are our best customers

In [0]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [0]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))


Add segment numbers to the RFM table

In [40]:
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
CustomerID,,,,,,
12747.0,3,7,358.56,3,4,2
12748.0,-2,544,3418.33,1,1,1
12826.0,-1,9,155.00,1,4,4
12838.0,7,59,390.79,4,1,2
12839.0,1,22,402.61,2,2,2


RFM segments split your customer base into an imaginary 3D cube. It is hard to visualize. However, we can sort it out.

Add a new column to combine RFM score, 111 is the highest score as we determined earlier.

In [41]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) \
                            + segmented_rfm.f_quartile.map(str) \
                            + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12747.0,3,7,358.56,3,4,2,342
12748.0,-2,544,3418.33,1,1,1,111
12826.0,-1,9,155.00,1,4,4,144
12838.0,7,59,390.79,4,1,2,412
12839.0,1,22,402.61,2,2,2,222


Apparently, the first customer is not our best customer at all.

Here is top 10 of our best customers!

In [42]:
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
15061.0,-2,109,17344.78,1,1,1,111
13777.0,0,35,6840.16,1,1,1,111
12748.0,-2,544,3418.33,1,1,1,111
17340.0,0,86,2671.27,1,1,1,111
13767.0,-2,39,1823.56,1,1,1,111
14667.0,-2,130,1685.70,1,1,1,111
15311.0,-1,99,1624.07,1,1,1,111
15514.0,-2,62,1578.19,1,1,1,111
17841.0,-1,186,1541.56,1,1,1,111
